# T5 RAG model

## data preparation

In [23]:
%pip install torch transformers faiss-cpu numpy rouge-score nltk sacrebleu
%pip install sentence-transformers
%pip install langchain
%pip install -U langchain-community


In [24]:
# import libraries
import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import faiss
import json
import time
import psutil
from functools import lru_cache
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [25]:
def clean_definitions(definitions):
    cleaned_definitions = []
    for definition in definitions:
        cleaned_definition = definition.strip().replace('\n', ' ').replace('&lt;', '<').replace('&gt;', '>')
        cleaned_definitions.append(cleaned_definition)
    return cleaned_definitions


In [26]:
# define class fit the format
class Document:
    def __init__(self, page_content, metadata=None, doc_id=None):
        self.page_content = page_content
        self.metadata = metadata or {}
        self.id = doc_id if doc_id is not None else hash(page_content)


# Load definition files
def load_definitions(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

## retrieval model

In [27]:
# Generate embeddings for definitions
class Retriever:
    def __init__(self, definitions, num_retrieved_docs=5):
        all_documents = [Document(definition, doc_id=str(i)) for i, definition in enumerate(definitions)]
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.db = FAISS.from_documents(all_documents, embeddings)
        self.retriever = self.db.as_retriever(search_kwargs={"k": num_retrieved_docs})
        self.index = self.db.index

    def search(self, query):
        return self.retriever.get_relevant_documents(query)

## generate model

In [28]:
models = ['t5-small', 'google/flan-t5-small','google/flan-t5-large','google/flan-t5-xxl']

# setup the generation model
class T5Assistant:
    def __init__(self, model_names):
        self.models = {}
        for model_name in model_names:
            print(f"Loading model: {model_name}")
            tokenizer = T5Tokenizer.from_pretrained(model_name)
            model = T5ForConditionalGeneration.from_pretrained(model_name)
            self.models[model_name] = {
                "tokenizer": tokenizer,
                "model": model
            }

    def create_prompt(self, query, retrieved_info):
        return (f"Explain the concept or answer the question in a detailed manner using simple words and examples.\n"
                f"Instruction: {query}\n"
                f"Relevant information: {retrieved_info}\n"
                f"Output:")

    def generalreplyies(self, query, retrieved_info):
        prompt = self.create_prompt(query, retrieved_info)
        results = {}

        for model_name, components in self.models.items():
            tokenizer = components["tokenizer"]
            model = components["model"]

            input_ids = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids
            outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            results[model_name] = generated_text

        return results




In [29]:
def cleanreplys(reply):
    cleaned_reply = reply.replace("Instruction:", "").replace("Relevant information:", "").strip()
    return cleaned_reply


## evaluation

In [30]:
def bleuresults(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    smoothie = SmoothingFunction().method4
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

def printmemorys():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Memory Usage: {memory_info.rss / 1024 / 1024:.2f} MB")


def memories():
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / 1024 / 1024

@lru_cache(maxsize=10)
def cached_generate_reply(assistant, query, retrieved_info):
    return assistant.generate_reply(query, retrieved_info)

In [31]:
if __name__ == "__main__":
    definitions = load_definitions('ctx_pd.txt')
    definitions = clean_definitions(definitions)

    # initialize the retriever and generate model
    retriever = Retriever(definitions, num_retrieved_docs=5)
    T5sassistance = T5Assistant(models)

    # save the result to a list and dictionary
    resultslist = []
    data_samples = {
        'question': [],
        'answer': [],
        'contexts': []
    }

    # sample queries
    generalquestions = [
        "What is the definition of Articulation?",
        "Which accreditation framework is mentioned under the definition of Doctoral Degree?",
        "Which organization is referenced in the definition of Assessment?",
        "What is the difference between a Bachelor Degree and a Bachelor Honours Degree with reference to qualification level in the AQF?",
        "If a student studies in a foreign educational institution and is not a citizen or permanent resident of Australia, what term would the dataset use to categorize them as per the definitions?",
        "If a student completes a Certificate III and intends to directly pursue a Bachelor Degree, which concept from this dataset would likely apply to their transition?",
        "Describe the relationship between Foundation Course and Pathway Course based on their respective definitions.",
        "What percentage of modules studied that received a pass grade is referred to in the dataset, and under what name is this metric captured?",
        "If a student qualifies under the Doctoral Degree (Research) category of the AQF, what learning outcome is significant in their qualification process?",
        "Based on the definitions provided, how would the process of Admission differ from the process of Application, and what criteria must a student meet to progress from one to the other?"
    ]

    # go over all the queries
    for query in generalquestions:
        start_time = time.time()

        # search the relevent definition
        retrieved_docs = retriever.search(query)
        retrinfo = " ".join([doc.page_content for doc in retrieved_docs[:3]])
        referencess = [doc.page_content for doc in retrieved_docs]

        # create replies for different models
        repliesgeneration = T5sassistance.generalreplyies(query, retrinfo)

        # go over the results
        for model_name, reply in repliesgeneration.items():
            reply = cleanreplys(reply)
            bleu_score = bleuresults(" ".join(referencess), reply) if referencess else "N/A"

            # record the memory usage
            memory_usage = memories()

            # calculate the time use
            end_time = time.time()
            time_taken = end_time - start_time

            # save the result to the list and dictionary
            resultslist.append({
                'Query': query,
                'Model': model_name,
                'Generated Reply': reply,
                'Reference Answer': " ".join(referencess),
                'BLEU Score': bleu_score,
                'Memory Usage (MB)': memory_usage,
                'Time Taken (seconds)': time_taken
            })

            data_samples['question'].append(query)
            data_samples['answer'].append(reply)
            data_samples['contexts'].append(referencess)

        printmemorys()
    # save the result to the csv and json
    csvresults = pd.DataFrame(resultslist)
    csvresults.to_csv('t5models.csv', index=False)

    print("Results saved to t5models.csv")
    with open('t5models.json', 'w') as f:
        json.dump(data_samples, f, indent=4)

    print("Data samples saved to t5models.json")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading model: t5-small
Loading model: google/flan-t5-small
Loading model: google/flan-t5-large
Loading model: google/flan-t5-xxl


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Memory Usage: 41575.90 MB
Memory Usage: 40501.29 MB
Memory Usage: 42134.40 MB
Memory Usage: 39075.36 MB
Memory Usage: 41125.88 MB
Memory Usage: 39928.55 MB
Memory Usage: 38316.47 MB
Memory Usage: 40971.36 MB
Memory Usage: 40230.39 MB
Memory Usage: 38578.72 MB
Results saved to t5models.csv
Data samples saved to t5models.json
